# Tutorial for Simple Route Optimization with ASKCOS 
Assumes ASKCOS exists in directory and can be imported. 
Follow installation instructions in README

### Import Targets and Scores

In [2]:
import pandas as pd 

target_csv = "target_list_2mols.csv"
df = pd.read_csv(target_csv)
target_dict = {smi: score for smi, score in zip(df['SMILES'], df['score'])}

### Build Tree 

In [6]:
import sys 
sys.path.append('/home/jfromer/sparrow/') # change this for your system to where askcos folder is located

from sparrow.tree_build_utils import build_rxn_graph
route_graph = build_rxn_graph(
    target_smis=list(target_dict.keys()),
    n_cpus=5,
    time_per_target=15
)

INFO@file_pricer         : [276.792s]	Loaded prices from flat file
INFO@chemhistorian       : [276.793s]	Loading chemhistorian from file...
INFO@chemhistorian       : [310.086s]	Historian is fully loaded.
INFO@template_transformer: [310.200s]	Loading templates from /home/jfromer/sparrow/askcos/data/db/templates/retro.templates.json.gz
INFO@template_transformer: [313.545s]	Loaded templates. Using 163723 templates
INFO@mcts_tree_builder   : [313.547s]	Doing a hard worker reset


### Set up the problem and solve 
First, define RouteSelector object and calculate reaction scores. Only reason to _not_ calculate scores is if you are loading a previously saved route_graph which has reaction scores already. (Reaction scores indicate how confident the ASCKOS forward predictor module is in desired product actually being produced)
Second, define optimization problem variables, set the objectives and constraints, and finally solve

In [5]:
from sparrow.route_selector import RouteSelector

route_selector = RouteSelector(
    route_graph,
    target_dict,
    calc_reaction_scores=True,
)

route_selector.define_variables()
route_selector.set_objective()
route_selector.set_constraints()
route_selector.optimize()



Target COc1cc(Nc2ncc(F)c(Nc3ccc4c(n3)N(COP(=O)(O)O)C(=O)C(C)(C)O4)n2)cc(OC)c1OC not tagged in route graph because it is not already a node
Reward not set for target COc1cc(Nc2ncc(F)c(Nc3ccc4c(n3)N(COP(=O)(O)O)C(=O)C(C)(C)O4)n2)cc(OC)c1OC because it is not already a node
INFO@fast_filter         : [255.850s]	Starting to load fast filter
INFO@fast_filter         : [256.337s]	Done loading fast filter
##### Unparseable mol: Nc1nc(Cl)c(F)c[nH]1
##### Unparseable mol: Nc1nc(Cl)c(F)c[nH]1
##### Unparseable mol: Nc1nc(Cl)c(F)c[nH]1
##### Unparseable mol: Nc1ncc(F)c[nH]1
##### Unparseable mol: O=P(Cl)Nc1nc(Cl)c(F)c[nH]1
##### Unparseable mol: O=P(Cl)Nc1nc(Cl)c(F)c[nH]1
##### Unparseable mol: O=P(Cl)Nc1nc(Cl)c(F)c[nH]1
##### Unparseable mol: O=P(Cl)Nc1nc(Cl)c(F)c[nH]1
##### Unparseable mol: O=P(Cl)Nc1nc(Cl)c(F)c[nH]1
##### Unparseable mol: O=P(Cl)Nc1nc(Cl)c(F)c[nH]1
##### Unparseable mol: O=P(Cl)(Cl)Nc1nc(Cl)c(F)c[nH]1
##### Unparseable mol: O=P(Cl)(Cl)Nc1nc(Cl)c(F)c[nH]1
##### Unparseable mol: 

KeyboardInterrupt: 

### Visualize the results 

In [ ]:
from sparrow.visualizer import Visualizer
import matplotlib.pyplot as plt

vis = Visualizer(
    route_graph,
    nonzero_vars=route_selector.optimal_variables(),
    )

fig_out_dir = "tutorials/askcos_simple.png"
vis.plot_graph(fig_out_dir)

plt.show()